In [4]:
##### choose model of:
# rattay_01
# frijns_94
# frijns_05
# smit_09
# smit_10
# imennov_09
# negm_14

model_name = "smit_10"

In [5]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Show/hide the code by clicking <a href="javascript:code_toggle()">here</a>.''')

In [6]:
%matplotlib inline

##### don't show warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

##### import packages
from brian2 import *
from brian2.units.constants import zero_celsius, gas_constant as R, faraday_constant as F
import numpy as np
import pandas as pd
pd.set_option('display.width', 115)
import matplotlib.pyplot as plt
import os
from IPython.display import Markdown as md

##### sets working directory
os.chdir("C:/Users/Richard/Documents/Studium/Master Elektrotechnik/Semester 4/Python/Models Brian2")

##### import models
import models.Rattay_2001 as rattay_01
import models.Frijns_1994 as frijns_94
import models.Briaire_2005 as briaire_05
import models.Smit_2009 as smit_09
import models.Smit_2010 as smit_10
import models.Imennov_2009 as imennov_09
import models.Negm_2014 as negm_14

##### makes code faster and prevents warning
prefs.codegen.target = "numpy"

# =============================================================================
# Load data
# =============================================================================
model = eval(model_name)

##### load dataframes for tables
strength_duration_data = pd.read_csv("test_battery_results/{}/Strength_duration_data {}.csv".format(model.display_name,model.display_name))
threshold_table = pd.read_csv("test_battery_results/{}/Threshold_table {}.csv".format(model.display_name,model.display_name))
relative_spreads = pd.read_csv("test_battery_results/{}/Relative_spreads {}.csv".format(model.display_name,model.display_name))
conduction_velocity_table = pd.read_csv("test_battery_results/{}/Conduction_velocity_table {}.csv".format(model.display_name,model.display_name))
single_node_response_deterministic  = pd.read_csv("test_battery_results/{}/Single_node_response_deterministic {}.csv".format(model.display_name,model.display_name))
single_node_response_stochastic = pd.read_csv("test_battery_results/{}/Single_node_response_stochastic {}.csv".format(model.display_name,model.display_name))
refractory_table = pd.read_csv("test_battery_results/{}/Refractory_table {}.csv".format(model.display_name,model.display_name))

##### get plot links
strength_duration_curve_link = f"test_battery_results/{model.display_name}/Strength_duration_curve {model.display_name}.png"
relative_spread_plot_link = f"test_battery_results/{model.display_name}/Relative_spreads_plot {model.display_name}.png"
single_node_response_link = f"test_battery_results/{model.display_name}/Single_node_response {model.display_name}.png"
refractory_curve_link = f"test_battery_results/{model.display_name}/Refractory_curve {model.display_name}.png"
post_stimulus_time_histogram_link = f"test_battery_results/{model.display_name}/PSTH {model.display_name}.png"

# =============================================================================
# Add experimental results to tables
# =============================================================================
##### Strength duration data
strength_duration_data = strength_duration_data.transpose()
strength_duration_data = strength_duration_data.rename(index = str, columns={0:"model"})
strength_duration_data["model"] = ["%.3g" %strength_duration_data["model"][i] for i in range(0,2)]
strength_duration_data["Van den Honert and Stypulkowski 1984"] = ["95.8", "247"]

##### Relative spread of thresholds
relative_spreads = relative_spreads.rename(index = str, columns={"relative spread":"model"})
relative_spreads["experiments"] = ["6.3%","5-10%","12%","11%"]
relative_spreads["reference"] = ["Miller et al. 1999","Dynes 1996","Javel et al. 1987","Javel et al. 1987"]
relative_spreads = relative_spreads.set_index(["phase duration (us)","pulse form"])

##### Conduction velocity
conduction_velocity_table = conduction_velocity_table.transpose()
conduction_velocity_table = conduction_velocity_table.rename(index = str, columns={0:"model"})
if hasattr(model, "index_soma"):
    conduction_velocity_table["Hursh 1939"] = ["-","-","6","-","-","-"]
    if model.dendrite_outer_diameter < 12*um:
        dendrite_ratio = 4.6
    else:
        dendrite_ratio = 5.66
    conduction_velocity_table["Boyd and Kalu 1979"] = ["-","-","{}".format(dendrite_ratio),"-","-","-"]
    conduction_velocity_table["Czèh et al 1976"] = ["-","-","-","v_ax = 0.9*v_den-6.9*m/s","-","-"]

else:
    conduction_velocity_table["Hursh 1939"] = ["-","-","6"]
    if model.fiber_outer_diameter < 12*um:
        dendrite_ratio = 4.6
    else:
        dendrite_ratio = 5.66
    conduction_velocity_table["Boyd and Kalu 1979"] = ["-","-","{}".format(dendrite_ratio)]

##### Latency and jitter
latency = single_node_response_deterministic[["phase duration (us)", "pulse form", "amplitude level", "latency (us)"]]
jitter = single_node_response_stochastic[["phase duration (us)", "pulse form", "amplitude level", "jitter (us)"]]
latency_jitter = pd.merge(single_node_response_deterministic, single_node_response_stochastic, on=["phase duration (us)","pulse form","amplitude level"], how="left")

latency_jitter = pd.melt(latency_jitter, id_vars=["phase duration (us)", "pulse form", "amplitude level"], value_vars=["latency (us)",  "jitter (us)"])
latency_jitter["value"] = ["%.3g" %latency_jitter["value"][i] for i in range(0,latency_jitter.shape[0])]
latency_jitter["phase duration (us)"] = ["%g us" %latency_jitter["phase duration (us)"][i] for i in range(0,latency_jitter.shape[0])]
latency_jitter = latency_jitter.rename(index = str, columns={"phase duration (us)":"phase duration"})

latency_jitter_th = latency_jitter[latency_jitter["amplitude level"] == "1*threshold"]
latency_jitter_th = latency_jitter_th.rename(index = str, columns={"value":"model (threshold)"})
latency_jitter_th = latency_jitter_th.drop(columns = ["amplitude level"])

latency_jitter_2th = latency_jitter[latency_jitter["amplitude level"] == "2*threshold"]
latency_jitter_2th = latency_jitter_2th.rename(index = str, columns={"value":"model (2*threshold)"})
latency_jitter_2th = latency_jitter_2th.drop(columns = ["amplitude level"])

latency_jitter = latency_jitter.drop(columns = ["amplitude level", "value"]).drop_duplicates()

latency_jitter = pd.merge(latency_jitter, latency_jitter_th, on=["phase duration","pulse form","variable"], how = 'left')
latency_jitter = pd.merge(latency_jitter, latency_jitter_2th, on=["phase duration","pulse form","variable"], how = 'left')

latency_jitter = latency_jitter.sort_values(by=["pulse form", "phase duration"], ascending = [False, True])

latency_jitter["Miller et al. 1999"] = ["650", "100", "-", "-", "-", "-", "-", "-"]
latency_jitter["Van den Honert and Stypulkowski 1984 (threshold)"] = ["-", "-", "-", "-", "685", "352", "-", "-"]
latency_jitter["Van den Honert and Stypulkowski 1984 (2*threshold)"] = ["-", "-", "-", "-", "352", "8", "-", "-"]
latency_jitter["Hartmann and al. 1984"] = ["-", "-", "-", "-", "-", "-", "300-400", "-"]
latency_jitter["Cartee et al. 2000 (threshold)"] = ["-", "-", "440", "80", "-", "-", "-", "-"]

latency_jitter = latency_jitter.rename(index = str, columns={"variable":"property"})
latency_jitter = latency_jitter.set_index(["phase duration","pulse form", "property"])

##### AP shape
AP_shape = single_node_response_deterministic[["AP height (mV)", "rise time (us)", "fall time (us)", "AP duration (us)"]].iloc[[5]].transpose()
AP_shape = AP_shape.rename(index = str, columns={5:"model"})

t_rise = round(-0.000625*conduction_velocity_table["model"].iloc[[0]] + 0.14, 3).tolist()[0]
t_fall = round(-0.002083*conduction_velocity_table["model"].iloc[[0]] + 0.3933, 3).tolist()[0]
AP_duration = t_rise + t_fall
AP_shape["Paintal 1966"] = ["-", int(t_rise*1e3), int(t_fall*1e3), int(AP_duration*1e3)]

##### Refractory periods
absolute_refractory_periods = refractory_table.drop(columns = ["relative refractory period (ms)"])
absolute_refractory_periods = absolute_refractory_periods.rename(index = str, columns={"absolute refractory period (us)":"ARP model (us)"})
absolute_refractory_periods["ARP model (us)"] = absolute_refractory_periods["ARP model (us)"].astype(int)
absolute_refractory_periods["ARP Experiments (us)"] = ["334","300","500-700","400-500","-"]
absolute_refractory_periods["reference"] = ["Miller et al. 2001","Stypulkowski and Van den Honert 1984","Dynes 1996","Brown and Abbas 1990", "-"]
absolute_refractory_periods = absolute_refractory_periods[absolute_refractory_periods["ARP Experiments (us)"] != "-"]
absolute_refractory_periods = absolute_refractory_periods.set_index(["phase duration (us)","pulse form"])

relative_refractory_periods = refractory_table.drop(columns = ["absolute refractory period (us)"])
relative_refractory_periods = relative_refractory_periods.rename(index = str, columns={"relative refractory period (ms)":"RRP model (ms)"})
relative_refractory_periods["RRP Experiments (ms)"] = ["-","3-4; 4-5","5","-","5"]
relative_refractory_periods["reference"] = ["-","Stypulkowski and Van den Honert 1984; Cartee et al. 2000","Dynes 1996","-", "Hartmann et al. 1984"]
relative_refractory_periods = relative_refractory_periods[relative_refractory_periods["RRP Experiments (ms)"] != "-"]
relative_refractory_periods = relative_refractory_periods.set_index(["phase duration (us)","pulse form"])


In [7]:
md("# %s: Test battery results"%(model.display_name))

# Smit et al. 2010: Test battery results

## 1. Strength duration curve

In [8]:
md("![title](%s?arg)"%(strength_duration_curve_link))

![title](test_battery_results/Smit et al. 2010/Strength_duration_curve Smit et al. 2010.png?arg)

In [9]:
strength_duration_data

,model,Van den Honert and Stypulkowski 1984
rheobase (uA),0.152,95.8
chronaxie (us),160,247


## 2. Relative spread of thresholds

In [10]:
md("![title](%s?arg)"%(relative_spread_plot_link))

![title](test_battery_results/Smit et al. 2010/Relative_spreads_plot Smit et al. 2010.png?arg)

In [11]:
relative_spreads

,,model,experiments,reference
phase duration (us),pulse form,,,
40,monophasic,10.66%,6.3%,Miller et al. 1999
100,monophasic,11.55%,5-10%,Dynes 1996
200,biphasic,14.42%,12%,Javel et al. 1987
400,biphasic,17.72%,11%,Javel et al. 1987


## 3. Conduction velocity

In [12]:
conduction_velocity_table

,model,Hursh 1939,Boyd and Kalu 1979,Czèh et al 1976
velocity dendrite (m/s),11.833,-,-,-
outer diameter dendrite (um),1.680,-,-,-
velocity/diameter dendrite,7.040,6,4.6,-
velocity axon (m/s),12.229,-,-,v_ax = 0.9*v_den-6.9*m/s
outer diameter axon (um),3.750,-,-,-
velocity/diameter axon,3.260,-,-,-


## 4. Single node response properties

In [13]:
md("![title](%s?arg)"%(single_node_response_link))

![title](test_battery_results/Smit et al. 2010/Single_node_response Smit et al. 2010.png?arg)

### 4.1 Action potential shape

In [14]:
AP_shape

,model,Paintal 1966
AP height (mV),134.0,-
rise time (us),96.0,133
fall time (us),1040.0,369
AP duration (us),1140.0,502


### 4.2 Latency and jitter

In [15]:
latency_jitter

model (threshold) model (2*threshold) Miller et al. 1999  \
phase duration pulse form property                                                                
100 us         monophasic latency (us)               nan                 nan                650   
                          jitter (us)               81.3                3.52                100   
40 us          monophasic latency (us)               nan                 nan                  -   
                          jitter (us)               62.6                3.31                  -   
50 us          monophasic latency (us)               nan                 nan                  -   
                          jitter (us)               60.2                3.64                  -   
200 us         biphasic   latency (us)               nan                 nan                  -   
                          jitter (us)               53.3               0.304                  -   

                                       Van den Honert and Stypulkowski 1984 (threshold)  \
phase duration pulse form property                                                        
100 us         monophasic latency (us)                                                -   
                          jitter (us)                                                 -   
40 us          monophasic latency (us)                                                -   
                          jitter (us)                                                 -   
50 us          monophasic latency (us)                                              685   
                          jitter (us)                                               352   
200 us         biphasic   latency (us)                                                -   
                          jitter (us)                                                 -   

                                       Van den Honert and Stypulkowski 1984 (2*threshold) Hartmann and al. 1984  \
phase duration pulse form property                                                                                
100 us         monophasic latency (us)                                                  -                     -   
                          jitter (us)                                                   -                     -   
40 us          monophasic latency (us)                                                  -                     -   
                          jitter (us)                                                   -                     -   
50 us          monophasic latency (us)                                                352                     -   
                          jitter (us)                                                   8                     -   
200 us         biphasic   latency (us)                                                  -               300-400   
                          jitter (us)                                                   -                     -   

                                       Cartee et al. 2000 (threshold)  
phase duration pulse form property                                     
100 us         monophasic latency (us)                              -  
                          jitter (us)                               -  
40 us          monophasic latency (us)                            440  
                          jitter (us)                              80  
50 us          monophasic latency (us)                              -  
                          jitter (us)                               -  
200 us         biphasic   latency (us)                              -  
                          jitter (us)                               -

## 5. Recfractory properties

In [16]:
md("![title](%s?arg)"%(refractory_curve_link))

![title](test_battery_results/Smit et al. 2010/Refractory_curve Smit et al. 2010.png?arg)

### 5.1 Absolute refractory periods

In [17]:
absolute_refractory_periods

,,ARP model (us),ARP Experiments (us),reference
phase duration (us),pulse form,,,
40,monophasic,1883,334,Miller et al. 2001
50,monophasic,1877,300,Stypulkowski and Van den Honert 1984
100,monophasic,1853,500-700,Dynes 1996
50,biphasic,1852,400-500,Brown and Abbas 1990


### 5.2 Relative refractory periods

In [18]:
relative_refractory_periods

,,RRP model (ms),RRP Experiments (ms),reference
phase duration (us),pulse form,,,
50,monophasic,2.326,3-4; 4-5,Stypulkowski and Van den Honert 1984; Cartee e...
100,monophasic,2.287,5,Dynes 1996
200,biphasic,2.003,5,Hartmann et al. 1984


## 6. Post-stimulus time histogramm

In [19]:
md("![title](%s?arg)"%(post_stimulus_time_histogram_link))

![title](test_battery_results/Smit et al. 2010/PSTH Smit et al. 2010.png?arg)

In [20]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border-right: 1px  black solid !important;
    text-align: center !important;
    color: black !important;
}